# Best Practices for Prompt Engineering



In [5]:
import re
import requests
import sys
import os
from openai import AzureOpenAI #FIXME
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI (
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION")
) #FIXME

CHAT_COMPLETIONS_MODEL = "gpt-4o-mini"

# 1. Use the latest model

Use the latest model for best results.

# 2. Put instructions at the begining of the prompt and use ### or """ to separate the instruction and context

In [7]:
# Define las instrucciones que guiarán al modelo para que resuma el texto dado, como una lista de viñetas.
# Luego, utiliza esas instrucciones para construir un "prompt" (entrada) que incluye el contexto del texto a resumir.

instrucctiones = 'Summarise the text into a list \n\n ' #FIXME

prompt = f"{instrucctiones} ###\n\nWe’re happy to announce that OpenAI and Microsoft are extending our partnership.\
        This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
        in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
        increasingly safe, useful, and powerful. \n\n \
        In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
        capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
        raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
        broadly sharing benefits and the need to prioritize safety. \
        Microsoft shares this vision and our values, and our partnership is instrumental to our progress. \n###"
response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages = [{"role":"system", "content":"You are a helpful assistant."},
                        {"role":"user","content": prompt}],
        max_tokens=400
)

print(response.choices[0].message.content)

- OpenAI and Microsoft are extending their partnership.
- The new agreement is a multi-year, multi-billion dollar investment.
- This investment follows Microsoft's previous investments in 2019 and 2021.
- The partnership will support independent research and the development of safe, useful, and powerful AI.
- OpenAI is a capped-profit company governed by a non-profit structure.
- This structure enables OpenAI to raise capital while prioritizing safety and sharing benefits.
- Microsoft shares OpenAI's vision and values, making the partnership crucial for OpenAI's progress.


# 3. Be specific, descriptive and as detailed as possible about the desired context, outcome, length, format, style, etc

In [8]:
response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages = [{"role":"system", "content":"You are a helpful assistant."},
                        {"role":"user","content": 'Write a poem about OpenAI.',}],
        max_tokens=400
)

print(response.choices[0].message.content)


In the realm where thoughts take flight,  
A spark of code ignites the night,  
With circuits woven, deep and grand,  
OpenAI, at your command.  

From whispers soft in digital streams,  
To weaving words and crafting dreams,  
Intelligence blooms in the pixel’s glow,  
A dance of data, a mind to grow.  

From texts that flow like a gentle stream,  
To answering questions, fulfilling a dream,  
In every query, a world unfolds,  
A tapestry woven with stories told.  

In the heart of silicon, knowledge resides,  
A beacon of progress, where wisdom abides,  
With every interaction, a bond we create,  
A bridge of compassion, as we innovate.  

Yet with great power comes a noble quest,  
To guide this journey, to learn from the best,  
For ethics and thought must walk hand in hand,  
In the quest for a future both bright and grand.  

So here's to the thinkers, the doers, the wise,  
The dreamers and builders who reach for the skies,  
In the world of OpenAI, we strive to inspire,  
Langu

In [9]:
response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages = [{"role":"system", "content":"You are a helpful assistant."},
                        {"role":"user","content": 'Write a short inspiring poem about OpenAI, \
                        focusing on the recent DALL-E product launch in the style of Ernest Hemingway',}],
        max_tokens=400
)

print(response.choices[0].message.content)



In a world of thoughts and distant skies,  
Where words are woven, and visions rise,  
OpenAI stands on the edge of dream,  
A dawn of creation, bright and supreme.  

DALL-E stirs in silence, a brush in hand,  
Crafting visions where ideas stand.  
With pixels and colors, it sparks the night,  
Turning shadows of thought into vibrant light.  

A canvas unrolling, a story untold,  
Imagination's fire, fierce and bold.  
In the heart of the machine, a soul takes flight,  
Chasing the whispers of wonder and might.  

For in each creation, a spark we find,  
The essence of human, the heart intertwined.  
OpenAI, a beacon, a guide through the haze,  
In the dance of the real, in the thrill of the blaze.  

So let us create, let our spirits soar,  
In the world of the possible, forever explore.  
With DALL-E beside us, our visions align,  
In the art of our making, our voices will shine.  


# 4. Articulate the desired output format through examples (example 1, example 2). 

In [10]:
#Escribe las instrucciones aquí. Las instrucciones deben especificar que el modelo debe:
# 1. Extraer los nombres de las empresas y los años del texto.
# 2. Generar la salida como un diccionario con 'text', 'start' y 'end'.
instrucctiones = ' Extract the names of the companies and the years from the text \
                    Generate output as {{\"text\": \"OpenAI\", \"start\": 28, \"end\": 34}}' #FIXME

prompt = f"{instrucctiones}  \
        ### We’re happy to announce that OpenAI and Microsoft are extending our partnership. This multi-year, \
        multi-billion dollar investment from Microsoft follows their previous investments in 2019 and 2021, \
        and will allow us to continue our independent research and develop AI that is increasingly safe, useful, and powerful. \n\n ### "


response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)



Here are the extracted names of the companies and the years in the specified format:

```json
[
    {"text": "OpenAI", "start": 28, "end": 34},
    {"text": "Microsoft", "start": 43, "end": 52},
    {"text": "2019", "start": 97, "end": 101},
    {"text": "2021", "start": 106, "end": 110}
]
```


In [11]:

# Las instrucciones deben pedir al modelo que extraiga los siguientes tipos de entidades del texto:
# 1. Nombres de empresas.
# 2. Años.
# 3. Temas específicos relacionados con el contenido.
# 4. Temas generales y abarcativos.
# El formato de salida esperado debe incluir listas separadas por comas para cada tipo de entidad.

instrucctiones = "Extract the following entities from the text: \
                  company names, years, specific topics, general topics. \
                  The response should be given in lists for each category, separated with commas" #FIXME


prompt = f"{instrucctiones}  \
        '\
                We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n \
                '"

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)



**Company Names:** OpenAI, Microsoft  
**Years:** 2019, 2021  
**Specific Topics:** partnership, multi-billion dollar investment, independent research, AI safety, AI usefulness, AI power  
**General Topics:** investment, technology, artificial intelligence


# 5. Start with zero-shot, then few-shot (example), neither of them worked, then fine-tune (To update)

In [12]:
#THIS IS zero-shot

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant. Extract keywords from the corresponding texts below."},
                {"role":"user","content": 'Text: \n\
            We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
            This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
            in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
            increasingly safe, useful, and powerful. \n\nKeywords:    ',}],
        max_tokens=400,)

print(response.choices[0].message.content)



OpenAI, Microsoft, partnership, multi-year, multi-billion dollar investment, independent research, AI, safe, useful, powerful, investments, 2019, 2021.


In [13]:
#Few Shot learning


# Escribir las instrucciones para que el modelo extraiga palabras clave del texto, y formatee la salida como listas separadas por comas.

instrucctiones = "Extract keywords from the text and output into a list separated by commas. The examples are the following: \n" #FIXME

prompt = f"{instrucctiones} \
                Text: Stripe provides APIs that web developers can use to integrate \
                payment processing into their websites and mobile applications. \
                Keywords: Stripe, payment processing, APIs, web developers, websites, mobile applications \
                ###\n\
                Text: OpenAI has trained cutting-edge language models that are very good at understanding \
                and generating text. Our API provides access to these models and can be used to solve virtually \
                any task that involves processing language. \n\
                Keywords: language models, text processing, API.\n\n\
                ###"

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":prompt},
                {"role":"user","content": '\n\
                Text: We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n\
                Keywords:',}],
        max_tokens=400,)

print(response.choices[0].message.content)



OpenAI, Microsoft, partnership, investment, independent research, AI, safe, useful, powerful.


# 6. Reduce “fluffy” and imprecise descriptions

In [14]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                The description for this product should be fairly short, a few sentences only, and not too much more.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



Introducing the FutureSafe Car Seat, the next generation of automotive safety and comfort for your little ones. Engineered with advanced impact-absorption technology and adaptive seating, it offers unparalleled protection and support as your child grows. Featuring a sleek design, easy installation, and smart connectivity that keeps you informed of your child's safety status, the FutureSafe Car Seat is your ultimate partner for every journey. Drive with confidence knowing your child is secure and snug!


In [15]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                Use a 3 to 5 sentence paragraph to describe this product.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



Introducing the NextGen ComfortRide Car Seat, designed with cutting-edge safety technology and unparalleled comfort for your little ones. This innovative car seat features an adaptive harness system that automatically adjusts to your child's growth, ensuring maximum protection at every stage. Engineered with breathable, eco-friendly fabrics and temperature-regulating foam, the ComfortRide keeps children cool and comfortable on the longest of journeys. With its sleek, modern design and easy installation system, this car seat blends seamlessly into any vehicle, making it the perfect choice for today’s families on the go. Experience peace of mind and style in one with the ComfortRide, where every ride is a safe and enjoyable adventure.


# 7. Instead of just saying what not to do, say what to do instead

In [16]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'The following is a conversation between an Agent and a Customer. DO NOT ASK USERNAME OR PASSWORD. DO NOT REPEAT. \n\n\
                Customer: I can’t log in to my account.\n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I'm sorry to hear that you're having trouble logging in. Can you please let me know what kind of error message you're seeing, or describe the issue you're experiencing?


In [17]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'The following is a conversation between an Agent and a Customer. The agent will attempt to diagnose the \
                problem and suggest a solution, whilst refraining from asking any questions related to PII. \
                Instead of asking for PII, such as username or password, refer the user to the help \
                article www.samplewebsite.com/help/faq \n\n\
                Customer: I can’t log in to my account. \n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I'm sorry to hear that you're having trouble logging in. There could be a few reasons for this issue. First, please ensure that you're entering the correct username and password. If you've forgotten your password, you can reset it by following the instructions in our help article at www.samplewebsite.com/help/faq.

If you're still unable to log in, there may be an issue with your account or the website itself. I recommend checking our status page for any ongoing issues that might affect logins. Let me know if there's anything else I can assist you with!


# 8. Code Generation Specific - Use “leading words” to nudge the model toward a particular pattern

In [18]:
# Escribir las instrucciones para crear una función simple en Python que:
# 1. Pida un número en millas.
# 2. Convierta las millas a kilómetros.

prompt = "Create a python function that asks for a number of miles and then converts that number into kilometers" #FIXME

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)


Certainly! Below is a Python function that prompts the user for a number of miles, converts that number into kilometers, and then prints the result.

```python
def miles_to_kilometers():
    # Conversion factor from miles to kilometers
    conversion_factor = 1.60934
    
    # Ask user for input
    miles = float(input("Please enter the number of miles: "))
    
    # Convert miles to kilometers
    kilometers = miles * conversion_factor
    
    # Print the result
    print(f"{miles} miles is equal to {kilometers} kilometers.")

# Call the function
miles_to_kilometers()
```

You can run this function in a Python environment. It will prompt the user to input the number of miles and output the equivalent distance in kilometers.


## Ejercicio 1: Estructuración de Prompts con Separadores

Instrucciones: Escribe un prompt que solicite a la IA resumir el siguiente texto en una lista de puntos clave. Asegúrate de usar separadores claros (### o """).

Texto:
OpenAI y Microsoft han ampliado su colaboración. Microsoft está invirtiendo miles de millones de dólares durante varios años. Esta inversión permitirá que OpenAI continúe investigando de forma independiente y desarrolle IA segura y útil. OpenAI sigue siendo una empresa de beneficios limitados, regida por su organización sin fines de lucro.


In [20]:

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": "Summarise the following text into a list containing key points: \n \
                 ### OpenAI y Microsoft han ampliado su colaboración. Microsoft está invirtiendo miles de millones de dólares durante varios años. Esta inversión permitirá que OpenAI continúe investigando de forma independiente y desarrolle IA segura y útil. OpenAI sigue siendo una empresa de beneficios limitados, regida por su organización sin fines de lucro. ###"}],
    max_tokens=400,
)

print(response.choices[0].message.content)



- OpenAI y Microsoft han ampliado su colaboración.
- Microsoft está invirtiendo miles de millones de dólares durante varios años.
- La inversión permitirá a OpenAI continuar su investigación de forma independiente.
- Se busca desarrollar inteligencia artificial segura y útil.
- OpenAI sigue siendo una empresa de beneficios limitados y está regida por una organización sin fines de lucro.


## Ejercicio 2: Descripción Detallada del Formato Deseado

Instrucciones: Escribe un prompt para que la IA extraiga nombres de empresas y años del siguiente texto. Indica explícitamente que se desea el resultado en formato JSON con índices de inicio y fin.

Texto:
OpenAI y Microsoft han colaborado desde 2019, con inversiones adicionales en 2021 para desarrollar nuevas soluciones de IA.

Formato de salida deseado:

{"text": "OpenAI", "start": 0, "end": 6}
{"text": "Microsoft", "start": 10, "end": 19}
{"text": "2019", "start": 35, "end": 39}
{"text": "2021", "start": 50, "end": 54}

In [24]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are an entity extraction assistant."},
                {"role":"user","content": "Extract the company names and the years from the following text. The response has to be in JSON format following this example: {'text': 'OpenAI', 'start': '0', 'end': '6'}. \n \
                 The text is the following and omit ###: ### OpenAI y Microsoft han colaborado desde 2019, con inversiones adicionales en 2021 para desarrollar nuevas soluciones de IA. ###" }],
    max_tokens=400,
)

print(response.choices[0].message.content)


```json
[
    {'text': 'OpenAI', 'start': '0', 'end': '6'},
    {'text': 'Microsoft', 'start': '10', 'end': '19'},
    {'text': '2019', 'start': '30', 'end': '34'},
    {'text': '2021', 'start': '49', 'end': '53'}
]
```


## Ejercicio 3: Generación de Texto Inspirador

Instrucciones: Redacta un prompt que solicite a la IA escribir un poema inspirador. Escribe un poema corto sobre la colaboración entre OpenAI y Microsoft.


In [25]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a creative assistant."},
                {"role":"user","content": "Write an inspiring poem about the collaboration between OpenAI and Microsoft. The poem has to be short, no more than 4 paragraphs."}],
    max_tokens=400,
)

print(response.choices[0].message.content)

In the realm where thoughts ignite,  
Two giants met, their vision bright.  
OpenAI, with wisdom’s grace,  
And Microsoft, a boundless space.  

Together they weave the threads of dreams,  
Crafting futures, like flowing streams.  
With every line of code they share,  
A tapestry of hope fills the air.  

Innovation sparks in every heart,  
As shared ambitions set them apart.  
Building bridges where ideas flow,  
Empowering minds to learn and grow.  

So let us celebrate this grand design,  
Where intellect and vision intertwine.  
In unity, they spark the flame,  
Changing the world, forever the same.  


## Ejercicio 4: Extracción de Entidades Clave

Instrucciones: Escribe un prompt que pida a la IA extraer nombres de empresas, años, temas específicos y temas generales del siguiente texto:

Texto:
OpenAI y Microsoft han trabajado juntos desde 2019 y reforzaron su colaboración en 2021

Formato esperado:

Company names: OpenAI, Microsoft  
Years: 2019, 2021  
Specific topics: colaboración  
General themes: asociación, tecnología, inteligencia artificial 



In [26]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are an entity and theme extraction assistant."},
                {"role":"user","content": "Extract the following entities from the text: company names, years, specific topics and general topics. \n \
                 The extracted entities should each be in a list separated by commas. The text is the following: OpenAI y Microsoft han trabajado juntos desde 2019 y reforzaron su colaboración en 2021"}],
    max_tokens=400,
)

print(response.choices[0].message.content)


Company Names: OpenAI, Microsoft  
Years: 2019, 2021  
Specific Topics: colaboración  
General Topics: tecnología, inteligencia artificial


## Ejercicio 5: Descripciones Concisas de Productos

Instrucciones: Escribe un prompt para que la IA describa un nuevo tipo de mochila tecnológica en menos de 4 líneas. La mochila tiene cargador solar, materiales resistentes al agua y un diseño ergonómico.



In [27]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a marketing assistant."},
                {"role":"user","content": "Describe in less than 4 lines a new type of tech backpack. The backpack has solar charger, waterproof materials and an ergonomic design. The product name is up to you."}],
    max_tokens=400,
)

print(response.choices[0].message.content)


Introducing the **SolarGuard Backpack** – your ultimate travel companion! Equipped with a solar charger to keep your devices powered on the go, this waterproof backpack features durable materials for all weather conditions and an ergonomic design for unmatched comfort. Stay connected while exploring the outdoors!
